In [ ]:
%load_ext pycodestyle_magic
# Add %%pycodestyle to the top of a cell to check it for PEP8 violations instead of running the code.
# Requires the pycodestyle and pycodestyle_magic modules from pip in your active env.

In [ ]:
# import foo
import re
from collections import Counter

In [ ]:
class Program:
    def __init__(self, name, weight, children):
        self.name = name
        self.weight = int(weight)
        if children:
            self.children = children.split(', ')
        else:
            self.children = None
        self.parent = None

    @property
    def cum_weight(self):
        if self.children:
            weight_children = sum(child.cum_weight for child in self.children)
        else:
            weight_children = 0
        return self.weight + weight_children

    def __str__(self):
        return "Name: {}, weight: {}, children: {}, cumulative weight: {}, parent: {}".format(
            self.name,
            self.weight,
            len(self.children) or '-',
            self.cum_weight,
            self.parent.name if self.parent else '-')


PROGRAM_PATTERN = re.compile(r'^(\w+)\s\((\d+)\)\s?-?>?\s?(.*)$')

In [ ]:
def calc_a(input):
    # init
    programs = {}
    for line in input:
        parsed_line = re.match(PROGRAM_PATTERN, line)
        line_program = Program(parsed_line.group(1),
                               parsed_line.group(2),
                               parsed_line.group(3) or None)
        programs[line_program.name] = line_program

    # Loop - build the tree
    for program in list(programs.values()):
        if program.children:
            children_programs = []
            for child in program.children:
                programs[child].parent = programs[program.name]
                children_programs.append(programs[child])
            program.children = children_programs

    # Find the root node of the tree
    # Pick a random node, and traverse up the tree
    random_program = list(programs.values())[0]
    while random_program.parent:
        random_program = random_program.parent

    return random_program


with open('data/7.txt', 'r') as input:
    answer = calc_a(input)
    print("Q7a: {}".format(answer))

In [ ]:
def calc_b(input):
    # init
    programs = {}
    for line in input:
        parsed_line = re.match(PROGRAM_PATTERN, line)
        line_program = Program(parsed_line.group(1),
                               parsed_line.group(2),
                               parsed_line.group(3) or None)
        programs[line_program.name] = line_program

    # Loop - build the tree
    for program in list(programs.values()):
        if program.children:
            children_programs = []
            for child in program.children:
                programs[child].parent = programs[program.name]
                children_programs.append(programs[child])
            program.children = children_programs

    # Find the root node of the tree
    # Pick a random node, and traverse up the tree
    random_program = list(programs.values())[0]
    while random_program.parent:
        random_program = random_program.parent
    root_node = random_program

    # Recursively traverse the tree in search of the imbalance
    new_weight, faulty_program = check_tree_weights(root_node)

    return new_weight, faulty_program


# Recursive function to traverse the faulty branches of the tree
def check_tree_weights(root):
    print("Traversing faulty (sub)tree for root program {}".format(root))
    if root.children:
        balance = Counter([c.cum_weight for c in root.children])
        # Find the odd one out among the child trees
        faulty_weight = [i for i, n in balance.items() if n == 1]
        if len(faulty_weight) == 1 and len(root.children) > 1:
            for c in root.children:
                if c.cum_weight == faulty_weight[0]:
                    # Traverse the sub-tree for faulty child c in search of imbalance
                    return check_tree_weights(c)
        else:
            print("All children of {} are in balance.".format(root.name))

        # If we get here, all children are in balance, or no children.
        # The current node is the issue of the imbalance.
        # Find the commonly shared cumulative weight among siblings
        balance = Counter([s.cum_weight for s in root.parent.children])
        good_weight = [i for i, n in balance.items() if n > 1][0]
        print("Expected proper cumulative weight: {}".format(good_weight))
        print("Actual cumulative weight: {}".format(root.cum_weight))
        print("We need to change Program {}'s weight from {} to {}".format(
            root.name,
            root.weight,
            root.weight + (good_weight-root.cum_weight))
            )
        return root.weight + (good_weight-root.cum_weight), root


with open('data/7.txt', 'r') as input:
    answer, faulty_program = calc_b(input)
    print("Q7b: new weight for program {} : {}".format(faulty_program, answer))
